# Evaluating Fine-Tuning vs. Prompting Techniques for Summarizing Albanian Parliamentary Speeches
In this notebook, we explore the efficacy of various Natural Language Processing (NLP) techniques for summarizing Albanian parliamentary speeches. The primary focus is on a comparative analysis between fine-tuning a Language Model (LM) and employing several prompt engineering strategies—specifically zero-shot, one-shot, and multi-shot approaches. The goal is to determine if prompt engineering, particularly multi-shot prompting, can provide a competitive alternative to the resource-intensive process of fine-tuning. After establishing the most effective prompting strategy, we further compare it to a fine-tuned model to ascertain which method yields superior summarization results for our specific dataset.
ine-tuning method.


## Zero-shot Prompt Engineering
Zero-shot prompting involves providing the model with no prior examples or context other than the task description itself. This approach tests the model's ability to generalize from its pre-trained knowledge without any task-specific tuning.

<img src="images/zero-shot.jpg" alt="Zero-shot Prompting Example" width="70%"/>

## One-shot Prompt Engineering
One-shot prompting provides the model with a single example of the task at hand before asking it to perform the task on new data. This method helps the model adjust its responses based on the context provided by one example.

<img src="images/one-shot.jpg" alt="One-shot Prompting Example" width="70%"/>

## Multi-shot Prompt Engineering
Multi-shot prompting provides the model with multiple examples of the task, offering more context and variations. This approach can potentially lead to better understanding and performance by showcasing different ways the task can be approached.

<img src="images/multi-shot.jpg" alt="Multi-shot Prompting Example" width="70%"/>


In [14]:
import os
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer

# Initialize OpenAI client with API key
api_key = os.getenv('OPENAI_API_KEY', 'sk-key-here')  # Replace with your API key
client = OpenAI(api_key=api_key)

# Load the sentence embedding model for similarity calculation
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# Function to generate a summary using OpenAI's GPT model with different shot settings
def generate_summary(text, example_texts=None, example_summaries=None, shot_type="zero-shot"):
    if shot_type == "zero-shot":
        prompt = f"Summarize the following Albanian parliamentary speech in Albanian: {text}"
    elif shot_type == "one-shot" and example_texts and example_summaries:
        prompt = f"Given this Albanian parliamentary speech text: '{example_texts[0]}' a good summary for it is this: '{example_summaries[0]}'. Now summarize the following Albanian parliamentary speech in Albanian: {text}"
    elif shot_type == "multi-shot" and example_texts and example_summaries:
        examples_prompt = " ".join([f"Given this Albanian parliamentary speech text: '{example_texts[i]}' a good summary for it is this: '{example_summaries[i]}'." for i in range(len(example_texts))])
        prompt = f"{examples_prompt} Now summarize the following Albanian parliamentary speech in Albanian: {text}"
    else:
        raise ValueError("Invalid shot type or insufficient examples provided for multi-shot or one-shot summarization.")

    try:
        completion = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500,
            temperature=0.5
        )
        return completion.choices[0].text.strip()
    except Exception as e:
        print(f"An error occurred during summary generation: {e}")
        return None

In [15]:
# Function to calculate cosine similarity and ROUGE scores between two texts
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
def calculate_scores(reference, summary):
    cosine_score = util.pytorch_cos_sim(
        embedding_model.encode(reference, convert_to_tensor=True),
        embedding_model.encode(summary, convert_to_tensor=True)
    ).item()
    rouge_scores = rouge.score(reference, summary)
    return cosine_score, rouge_scores

In [12]:
# Interaction for multi-shot prompting
def get_multiple_examples():
    num_examples = int(input("Enter the number of examples you want to provide: "))
    example_texts = []
    example_summaries = []
    for i in range(num_examples):
        print(f"Enter example text {i+1}:")
        example_text = input()
        print(f"Enter the summary for example text {i+1}:")
        example_summary = input()
        example_texts.append(example_text)
        example_summaries.append(example_summary)
    return example_texts, example_summaries
    
    # Example for multi-shot
example_texts, example_summaries = get_multiple_examples()

# Generate summaries
print("\nGenerating summaries with different approaches...")
zero_shot_summary = generate_summary(example_texts[0], shot_type="zero-shot")
one_shot_summary = generate_summary(example_texts[0], [example_texts[0]], [example_summaries[0]], shot_type="one-shot")
multi_shot_summary = generate_summary(example_texts[0], example_texts, example_summaries, shot_type="multi-shot")

Enter the number of examples you want to provide:  2


Enter example text 1:


 Faleminderit, kryetar! Pyetja është deponuar dy muaj më parë dhe përfshin tre ministra, në fakt është një për ministrin Lekaj, një për ministren Reshitaj dhe një për ministrin Gashi lidhur me vizitën e tyre në Gjakovë më 7 shkurt 2018. Ministër, gjatë kësaj vizite ju para medieve keni deklaruar, mes tjerash, se do t’i ndihmoni Komunës së Gjakovës në projektin e stërzgjatur të impiantit të ujërave të zeza, projekt ky me një vlerë prej 16 milionë euro. Pyetja është: Buxheti i miratuar i Kosovës për vitin 2018 në ministrinë tuaj përmban investime nga ministria për impiantin e ujërave të zeza në Prishtinë, 2 milionë e 700 mijë euro investim; impiantin e ujërave të zeza në Pejë, 3 milionë e 800 mijë euro investim, por nuk ka asnjë vijë buxhetore për projektin e impiantit të ujërave të zeza në Gjakovë. Nga cila kategori, ministre, do t’i ndihmoni ndërtimit të impiantit të ujërave të zeza të Gjakovës dhe nëse do t’i ndihmoni, me çfarë vlere është ajo ndihmë?


Enter the summary for example text 1:


 Pyetja është ngritur dy muaj më parë dhe përfshin tre ministra. Pyetja është: Buxheti i miratuar i Kosovës për vitin 2018 në ministrinë tuaj përmban investime nga ministria për impiantin e ujërave të zeza në Prishtinë, 2 milionë e 700 mijë euro investim. Por nuk ka asnjë linjë buxhetore për projektin e impiantit të ujërave të zeza në Gjakovë. Nga cila kategori, ministër, do ta ndihmoni ndërtimin e kanalizimeve të ujërave të zeza në Gjakovë dhe nëse do t'i ndihmoni, çfarë vlere ka kjo ndihmë?


Enter example text 2:


 Faleminderit kryetar! Po mendoj se ne anëtarët e Komisionit duhet të jemi më të përgjegjshëm nga vetë fakti se kemi marrë pjesë në intervistë dhe i kemi intervistuar 57 kandidatë. Procedura se si duhet të votohet, do të thotë ka qenë një procedurë e pavarur nga secili anëtar i Komisionit por nga vetë fakti se ky proces është përmbyllur me nënshkrimin e të gjithë anëtarëve të Komisionit lë të kuptohet se ne duhet të vazhdojmë sot apo duhet të fillojmë me përmbylljen e këtij procesi dhe mbetet në çështjen e deputetëve se si të votojnë për kandidatët. Unë mendoj se ka kandidatë këtu që janë kredibilë, ka kandidatë që e kanë dëshmuar veten atë ditë në intervistë, kandidatë që kanë edhe përvojë dhe mbetet që deputetët e Parlamentit të Kosovës të vlerësojnë secilin kandidat që të jetë anëtar i Bordit duke marrë për bazë, sikur që e tha edhe kolegia deputete se i kemi do afate që ne në shtator duhet t’i zëmë këto afate dhe njëkohësisht RTK-ja duhet t’i ketë anëtarët e Bordit. Mendoj se kjo m

Enter the summary for example text 2:


 Mendoj se ne anëtarët e Komisionit duhet të jemi më të përgjegjshëm për vetë faktin që morëm pjesë në intervistë dhe intervistuam 57 kandidatë. Mendoj se kjo mbetet në çështjen e deputetëve dhe kërkoj që votimi të bëhet pas një pushimi, nëse është e mundur, sepse siç shihet nuk ka as deputetë të mjaftueshëm, por këtë le ta vlerësojë kryesia e Kuvendit. Faleminderit!“Kemi dy afate që duhet t'i përmbushim këto afate në shtator dhe në të njëjtën kohë RTK duhet të ketë anëtarët e Bordit.



Generating summaries with different approaches...


In [16]:
print("\nCalculating scores...")
zero_cosine, zero_rouge = calculate_scores(example_summaries[0], zero_shot_summary)
one_cosine, one_rouge = calculate_scores(example_summaries[0], one_shot_summary)
multi_cosine, multi_rouge = calculate_scores(example_summaries[0], multi_shot_summary)


Calculating scores...


In [17]:
# Display results
print("\nSimilarity Metrics")
print("Zero-shot Summary:", zero_shot_summary)
print("Cosine Similarity:", zero_cosine)
print("ROUGE Scores:", zero_rouge)
print("One-shot Summary:", one_shot_summary)
print("Cosine Similarity:", one_cosine)
print("ROUGE Scores:", one_rouge)
print("Few-shot Summary:", multi_shot_summary)
print("Cosine Similarity:", multi_cosine)
print("ROUGE Scores:", multi_rouge)


Similarity Metrics
Zero-shot Summary: Kryetar, faleminderit! Pyetja është paraqitur dy muaj më parë dhe ka të bëjë me tre ministra, konkretisht për ministrin Lekaj, ministrin Reshitaj dhe ministrin Gashi, në lidhje me vizitën e tyre në Gjakovë më 7 shkurt 2018. Ministër, gjatë kësaj vizite, ju keni deklaruar para medieve se do të ndihmonit Komunën e Gjakovës në projektin e planifikuar të impiantit të ujërave të zeza, me një vlerë prej 16 milionë euro. Pyetja është: Buxheti i miratuar i Kosovës për vitin 2018 për ministrinë tuaj përfshin investime për impiantin e ujërave të zeza në Prishtinë, me një investim prej 2 milionë e 700 mijë euro; impiantin e ujërave të zeza në Pejë, me një investim prej 3 milionë e 800 mijë euro, por nuk ka asnjë linjë buxhetore për projektin e impiantit të ujërave të zeza në Gjakovë. Nga cila kategori, ministre, do të ndihmoni në ndërtimin e impiantit të ujërave të zeza në Gjakovë dhe nëse do të ndihmoni, me çfarë vlerë do të jetë ajo ndihmë?
Cosine Similari

### Finding the best Hyperparameters for Multishot prompt engineering

In [40]:
data = pd.read_excel('labeled_data.xlsx').sample(20)

In [41]:
import pandas as pd
import itertools
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

# Initialize OpenAI client with API key
api_key = os.getenv('OPENAI_API_KEY', 'sk-key-here')  # Replace with your API key
client = OpenAI(api_key=api_key)

# Load the sentence embedding model for similarity calculation
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to construct prompt with N examples
def construct_prompt(examples, current_text):
    prompt = "Summarize this Albanian text based on the following examples:\n"
    for idx, example in examples.iterrows():
        prompt += f"Text: {example['text']} Summary: {example['summarization']}\n"
    prompt += f"Text: {current_text} Summary:"
    return prompt

# Function to call the model
def generate_summary(prompt, temperature):
    try:
        completion = client.completions.create(
            model="gpt-3.5-turbo-instruct",  # Adjust model as necessary
            prompt=prompt,
            temperature=temperature,
            max_tokens=150
        )
        return completion.choices[0].text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to evaluate summaries
def calculate_cosine_similarity(reference, summary):
    ref_embedding = embedding_model.encode(reference, convert_to_tensor=True)
    summary_embedding = embedding_model.encode(summary, convert_to_tensor=True)
    cosine_score = util.pytorch_cos_sim(ref_embedding, summary_embedding).item()
    return cosine_score

# Iterate over combinations of number of examples and temperature
example_counts = [1, 2, 3]  # Number of examples to try
temperatures = [0.3, 0.5, 0.7]  # Different temperatures to try

best_score = 0
best_params = None

for count, temp in itertools.product(example_counts, temperatures):
    examples = data.sample(n=count)
    scores = []
    rouge_scores = []

    for _, current_example in data.iterrows():
        prompt = construct_prompt(examples, current_example['text'])
        generated_summary = generate_summary(prompt, temp)
        score = calculate_cosine_similarity(current_example['summarization'], generated_summary)
        scores.append(score)
        # Calculate ROUGE scores
        rouge_result = scorer.score(current_example['summarization'], generated_summary)
        rouge_scores.append(rouge_result)
    
    average_score = sum(scores) / len(scores)
    average_rouge = {key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores) for key in rouge_scores[0]}
    
    if average_score > best_score:
        best_score = average_score
        best_params = (count, temp, average_rouge)
        
    print(f"Tested {count} examples with temperature {temp}: Average Cosine Similarity = {average_score}, Average ROUGE Scores = {average_rouge}")

print(f"Best parameters are {best_params} with a cosine score of {best_score}")


E:\binawork\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tested 1 examples with temperature 0.3: Average Cosine Similarity = 0.8226228654384613, Average ROUGE Scores = {'rouge1': 0.48343722865411226, 'rouge2': 0.2538380629708836, 'rougeL': 0.3386794047338434}
Tested 1 examples with temperature 0.5: Average Cosine Similarity = 0.8113872706890106, Average ROUGE Scores = {'rouge1': 0.48163576039478545, 'rouge2': 0.23428479418275056, 'rougeL': 0.317383182677783}
Tested 1 examples with temperature 0.7: Average Cosine Similarity = 0.8145684063434601, Average ROUGE Scores = {'rouge1': 0.4660504852355472, 'rouge2': 0.21347260353029268, 'rougeL': 0.30695809446267674}
Tested 2 examples with temperature 0.3: Average Cosine Similarity = 0.8260851621627807, Average ROUGE Scores = {'rouge1': 0.5064617248631298, 'rouge2': 0.26104423173957536, 'rougeL': 0.3441954067356462}
Tested 2 examples with temperature 0.5: Average Cosine Similarity = 0.8329042851924896, Average ROUGE Scores = {'rouge1': 0.5088764204084966, 'rouge2': 0.26502297368212363, 'rougeL': 0.34

## Conclusion and Comparative Analysis

After applying different strategies for summarizing Albanian parliamentary speeches, we have gathered significant insights into the effectiveness of multishot prompt engineering compared to the traditional approach of fine-tuning a language model.

### Multishot Prompt Engineering Performance
- **Test Setup**: Employed 3 examples with a temperature setting of 0.3.
- **Performance Metrics**:
  - **Average Cosine Similarity**: 0.8543, indicating a high degree of semantic similarity between the generated summaries and the reference texts.
  - **ROUGE Scores**:
    - ROUGE-1: 0.5758
    - ROUGE-2: 0.3590
    - ROUGE-L: 0.4100
  These scores reflect good overlap with the reference summaries, especially considering the complexity of the summarization task.

### Fine-Tuning Model Performance
- **Performance Metrics**:
  - **Average Cosine Similarity**: 0.4731, which is significantly lower compared to the multishot approach.
  - **ROUGE Scores**:
    - ROUGE-1: 0.4769
    - ROUGE-2: 0.2626
    - ROUGE-L: 0.4460
  While the fine-tuned model shows competent summarization capabilities, it falls short of the multishot approach in terms of semantic similarity.

### Final Assessment
The comparison clearly indicates that **multishot prompt engineering not only outperforms the fine-tuned model in terms of similarity measures but also maintains competitive ROUGE scores**. This suggests that multishot prompting, despite being a less resource-intensive method, can be an effective strategy for tasks like summarizing legislative proceedings, offering a promising alternative to the more costly and computationally demanding fine-tuning process.
